In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/water-potability/water_potability.csv")
print(df.shape)
df.head()


In [ ]:
df.info()

## EDA

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df['ph'].plot(kind = 'hist')

In [ ]:
df[df['ph'].isna()]['Potability'].value_counts()

In [ ]:
#df['ph'].fillna(df['ph'].median() , inplace = True) 
#df['Sulfate'].fillna(df['Sulfate'].median() , inplace = True) 
#df['Trihalomethanes'].fillna(df['Trihalomethanes'].median() , inplace = True) 

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df['ph'].plot(kind = 'hist')

In [ ]:
## Potability :
print(df['Potability'].value_counts()/len(df))
df['Potability'].value_counts().plot(kind = 'bar')

In [ ]:
df.corr()

In [ ]:
X = df.drop(['Potability'] ,axis =1 )
Y = df['Potability']

from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.25, random_state = 49) 

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_std = sc.fit_transform(x_train)
x_test_std = sc.transform(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
def modelResult(model , x_train ,y_train ,  x_test ,y_test ):
    model.fit(x_train, y_train)
    print("Training Accuracy " , model.score(x_train, y_train))
    print("Test Accuracy" , model.score(x_test, y_test))
    predict = model.predict(x_test)
    print(confusion_matrix(predict, y_test))
    print(classification_report(predict, y_test))

## Logisitic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
modelResult(lr , x_train ,y_train ,  x_test ,y_test )
modelResult(lr , x_train_std ,y_train ,  x_test_std ,y_test )

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
modelResult(knn , x_train_std ,y_train ,  x_test_std ,y_test)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb =GaussianNB()
modelResult(nb , x_train ,y_train ,  x_test ,y_test)

## SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC(C = 5 , gamma = 'auto')
modelResult(svm , x_train_std ,y_train ,  x_test_std ,y_test)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth = 15 ,min_samples_split = 2 )
modelResult(dt , x_train ,y_train ,  x_test ,y_test)

In [ ]:
dt.feature_importances_

In [ ]:
feature_importance_df = pd.DataFrame()

feature_importance_df['Feature'] = X.columns
feature_importance_df['Feature_importance'] = dt.feature_importances_
feature_importance_df.sort_values(by = 'Feature_importance' , ascending= False)

## Ensemble : Bagging 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth= 12 , max_features= 3)
modelResult(rfc , x_train ,y_train ,  x_test ,y_test)


In [ ]:
from sklearn.ensemble import BaggingClassifier
bgc = BaggingClassifier(DecisionTreeClassifier(max_depth = 15 ,min_samples_split = 2 ) , )
modelResult(bgc , x_train ,y_train ,  x_test ,y_test)


## Ensemble Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier , GradientBoostingClassifier 
adc = AdaBoostClassifier()
gbc = GradientBoostingClassifier(n_estimators = 1000 )

modelResult(adc , x_train ,y_train ,  x_test ,y_test)

In [ ]:
modelResult(gbc , x_train ,y_train ,  x_test ,y_test)